In [1]:
import cv2
import numpy as np
import os
import skvideo.io
import video_tools
from moviepy.editor import VideoFileClip, clips_array
import anz
from matplotlib import colors

In [2]:
os.getcwd()

'C:\\Users\\Peter\\Desktop\\behavioral_analysis\\notebooks'

In [47]:
analyzed_dir = r'C:\Users\Peter\Desktop\DATA\M9\2021.03.09\ANALYZED'
d = os.path.join(analyzed_dir, 'VIDEOS_2D')
ns = ['M9_2021.03.09_00004_CAM0.mp4', 'M9_2021.03.09_00004_CAM1.mp4']
videos = [os.path.join(d, n) for n in ns]
output_folder = os.path.join(os.getcwd(), 'OUTPUTS', 'LABELLED_VIDEO')
output_name = os.path.join(output_folder, ns[0][:-8])
os.makedirs(output_folder, exist_ok=True)

cutoff_frames = 110
frame_rate = 40 ### not the actual frame rate, just what is recorded in video
seconds = cutoff_frames / frame_rate
clip = VideoFileClip(videos[0]).subclip(0, seconds)
clip1 = VideoFileClip(videos[1]).subclip(0, seconds)
final_clip = clips_array([[clip, clip1]])
final_clip.write_videofile(output_name + '.avi',
                           codec='mjpeg',
                           ffmpeg_params=['-pix_fmt', 'yuv420p',
                                          '-qscale', '0',
                                          ])

t:   2%|▏         | 2/110 [00:00<00:07, 14.60it/s, now=None]

Moviepy - Building video C:\Users\Peter\Desktop\behavioral_analysis\notebooks\OUTPUTS\LABELLED_VIDEO\M9_2021.03.09_00004_.avi.
Moviepy - Writing video C:\Users\Peter\Desktop\behavioral_analysis\notebooks\OUTPUTS\LABELLED_VIDEO\M9_2021.03.09_00004_.avi



Moviepy - Done !
Moviepy - video ready C:\Users\Peter\Desktop\behavioral_analysis\notebooks\OUTPUTS\LABELLED_VIDEO\M9_2021.03.09_00004_.avi


In [40]:
label_regions_per_video, label_names = anz._get_labels(
    os.path.join(analyzed_dir, 'LABELS'))

labels = ['extend', 'grab', 'supinate', 'chew']
labels_to_color = {'extend': 'red',
                   'grab': 'green',
                   'supinate': 'orange',
                   'chew': 'green'}
labels_to_color = {k: (255 * np.array(colors.to_rgb(v))).astype(int) for k, v in
                   labels_to_color.items()}

multiple = 30
labels_yc = {x: 80 + i*multiple for i, x in enumerate(labels)}

label_region = label_regions_per_video[4]
for k, v in label_region.items():
    temp = []
    for i, pair in enumerate(v):
        temp.append(list(range(pair[0], min(pair[1], cutoff_frames))))
    temp = [x for y in temp for x in y]
    label_region[k] = temp

In [48]:
# Create a VideoCapture object
cap = cv2.VideoCapture(output_name)
fps = cap.get(cv2.CAP_PROP_FPS)
frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

writer = skvideo.io.FFmpegWriter(output_name + '_labeled.avi',
                                 inputdict={
                                     '-hwaccel': 'auto',
                                     '-framerate': str(200),
                                 },
                                 outputdict={
                                     '-vcodec': 'mjpeg',
                                     '-qp': '25',
                                     '-qscale': '0',
                                     '-pix_fmt': 'yuv420p',
                                 })

font = cv2.FONT_HERSHEY_SIMPLEX
for i in range(frames):
    ret, frame = cap.read()
    if not ret:
        break
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = cv2.putText(img,
                      str(i),
                      (40, 40),
                      font,
                      1,
                      (255, 255, 255),
                      2,
                      cv2.LINE_4)
    x = 20
    for k, v in label_region.items():
        if i in v and k in labels:
            y = labels_yc[k]
            c = labels_to_color[k]

            img = cv2.rectangle(img,
                                (x, y-25),
                                (x+140, y+10),
                                (int(c[0]), int(c[1]), int(c[2])),
                                cv2.FILLED)
            img = cv2.putText(img,
                              k,
                              (x, y),
                              font,
                              1,
                              (255, 255, 255),
                              2,
                              cv2.LINE_8)
    writer.writeFrame(img)
cap.release()
writer.close()
cv2.destroyAllWindows()